In [1]:
%pip install duckduckgo_search


Looking in indexes: https://mirrors.aliyun.com/pypi/simple/Note: you may need to restart the kernel to use updated packages.



In [21]:
from config import *
from typing import List
from zigent.agents import ABCAgent, BaseAgent
from zigent.llm.agent_llms import LLM
from zigent.commons import TaskPackage
from zigent.actions.BaseAction import BaseAction
from zigent.logging.multi_agent_log import AgentLogger
from duckduckgo_search import DDGS

In [22]:
llm = LLM(api_key=api_key, base_url=base_url, model_name=chat_model)
response = llm.run("2023 中国国际智能产业博览会主题为")
print(response)

2023年中国国际智能产业博览会的主题是“智能化：为经济赋能，为生活添彩”。这个主题旨在强调智能化技术在推动经济发展和提升人民生活水平方面的关键作用。


In [4]:
class DuckSearchAction(BaseAction):
    def __init__(self) -> None:
        action_name = "DuckDuckGo_Search"
        action_desc = "Using this action to search online content."
        params_doc = {"query": "the search string. be simple."}
        self.ddgs = DDGS()
        super().__init__(
            action_name=action_name, 
            action_desc=action_desc, 
            params_doc=params_doc,
        )

    def __call__(self, query):
        results = self.ddgs.chat(query)
        return results

In [6]:
search_action = DuckSearchAction()
results = search_action("什么是 agent")
print(results)

“Agent”这个词在不同的领域有不同的含义。以下是一些常见的解释：

1. **一般意义**：在日常用语中，agent指的是一个代理人或代表，负责代表他人进行某种活动或决策。

2. **计算机科学**：在人工智能和计算机科学中，agent通常指的是一种能够自主执行任务的程序或系统。例如，智能代理（intelligent agent）可以根据环境的变化做出决策。

3. **商业**：在商业领域，agent可以指代中介或代理商，他们代表公司或个人进行交易或谈判。

4. **生物学**：在生物学中，agent可以指代某种能够引起生物反应的物质，如病原体或药物。

5. **法律**：在法律上，agent是指被授权代表他人行事的人，通常涉及合同或法律事务。

具体的含义取决于上下文。如果你有特定的领域或上下文，请告诉我，我可以提供更详细的信息。


In [7]:
class DuckSearchAgent(BaseAgent):
    def __init__(
        self,
        llm: LLM,
        actions: List[BaseAction] = [DuckSearchAction()],
        manager: ABCAgent = None,
        **kwargs
    ):
        name = "duck_search_agent"
        role = "You can answer questions by using duck duck go search content."
        super().__init__(
            name=name,
            role=role,
            llm=llm,
            actions=actions,
            manager=manager
        )

In [17]:
def do_search_agent():
    # 创建代理实例
    search_agent = DuckSearchAgent(llm=llm)

    # 创建任务
    task = "2024诺贝尔奖得主"
    task_pack = TaskPackage(instruction=task)

    # 执行任务并获取响应
    response = search_agent(task_pack)
    print("response:", response)

if __name__ == "__main__":
    do_search_agent()

Agent duck_search_agent receives the following TaskPackage:
[
	Task ID: 7668636c-0cf3-481a-9f3a-eba8d1b7d027
	Instruction: 2024诺贝尔奖得主
]
====duck_search_agent starts execution on TaskPackage 7668636c-0cf3-481a-9f3a-eba8d1b7d027====
Agent duck_search_agent takes 0-step Action:
{
	name: DuckDuckGo_Search
	params: {'query': '2024诺贝尔奖得主'}
}
Observation: 截至我最后的知识更新（2023年10月），2024年诺贝尔奖的得主尚未公布。诺贝尔奖通常在每年的10月宣布，涵盖多个领域，包括物理、化学、医学、文学、和平和经济学。如果你想了解过去的诺贝尔奖得主或相关[TLDR]
Agent duck_search_agent takes 1-step Action:
{
	name: Finish
	params: {'response': '根据我最后的知识更新，2024年诺贝尔奖的得主尚未公布。诺贝尔奖通常在每年的10月宣布，涵盖多个领域。如果你需要了解过去诺贝尔奖得主的信息，我可以帮你查询。'}
}
Observation: Task Completed.
=========duck_search_agent finish execution. TaskPackage[ID:7668636c-0cf3-481a-9f3a-eba8d1b7d027] status:
[
	completion: completed
	answer: 根据我最后的知识更新，2024年诺贝尔奖的得主尚未公布。诺贝尔奖通常在每年的10月宣布，涵盖多个领域。如果你需要了解过去诺贝尔奖得主的信息，我可以帮你查询。
]
response: 根据我最后的知识更新，2024年诺贝尔奖的得主尚未公布。诺贝尔奖通常在每年的10月宣布，涵盖多个领域。如果你需要了解过去诺贝尔奖得主的信息，我可以帮你查询。


这种方法查不了新闻吗

In [18]:
%pip install --upgrade zhipuai

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [23]:
import os
import json
import requests
from dotenv import load_dotenv
from typing import List
from zigent.agents import BaseAgent, ABCAgent
from zigent.llm.agent_llms import LLM
from zigent.commons import TaskPackage
from zigent.actions.BaseAction import BaseAction
from zhipuai import ZhipuAI
from datetime import datetime

ZHIPU_API_KEY = os.getenv('API_KEY')

# 配置LLM
llm = LLM(api_key=api_key, base_url=base_url, model_name=chat_model)

In [24]:
# 定义Zhipu Web Search工具
def zhipu_web_search_tool(query: str) -> str:
    """
    使用智谱AI的GLM-4模型进行联网搜索，返回搜索结果的字符串。
    
    参数:
    - query: 搜索关键词

    返回:
    - 搜索结果的字符串形式
    """
    # 初始化客户端
    client = ZhipuAI(api_key=ZHIPU_API_KEY)

    # 获取当前日期
    current_date = datetime.now().strftime("%Y-%m-%d")

    print("current_date:", current_date)
    
    # 设置工具
    tools = [{
        "type": "web_search",
        "web_search": {
            "enable": True
        }
    }]

    # 系统提示模板，包含时间信息
    system_prompt = f"""你是一个具备网络访问能力的智能助手，在适当情况下，优先使用网络信息（参考信息）来回答，
    以确保用户得到最新、准确的帮助。当前日期是 {current_date}。"""
        
    # 构建消息
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ]
        
    # 调用API
    response = client.chat.completions.create(
        model="glm-4-flash",
        messages=messages,
        tools=tools
    )
    
    # 返回结果
    return response.choices[0].message.content

In [25]:
class ZhipuSearchAction(BaseAction):
    def __init__(self) -> None:
        action_name = "Zhipu_Search"
        action_desc = "Using this action to search online content."
        params_doc = {"query": "the search string. be simple."}
        super().__init__(
            action_name=action_name,
            action_desc=action_desc,
            params_doc=params_doc,
        )

    def __call__(self, query):
        results = zhipu_web_search_tool(query)
        return results

In [27]:
class ZhipuSearchAgent(BaseAgent):
    def __init__(
        self,
        llm: LLM,
        actions: List[BaseAction] = [ZhipuSearchAction()],
        manager: ABCAgent = None,
        **kwargs
    ):
        name = "zhiu_search_agent"
        role = "You can answer questions by using Zhipu search content."
        super().__init__(
            name=name,
            role=role,
            llm=llm,
            actions=actions,
            manager=manager,
        )


In [29]:
def do_search_agent():
    # 创建代理实例
    search_agent = ZhipuSearchAgent(llm=llm)

    # 创建任务
    task = "2024诺贝尔奖得主"
    task_pack = TaskPackage(instruction=task)

    # 执行任务并获取响应
    response = search_agent(task_pack)
    print(response)

if __name__ == "__main__":
    do_search_agent()

Agent zhiu_search_agent receives the following TaskPackage:
[
	Task ID: 7668636c-0cf3-481a-9f3a-eba8d1b7d027
	Instruction: 2024诺贝尔奖得主
]
====zhiu_search_agent starts execution on TaskPackage 7668636c-0cf3-481a-9f3a-eba8d1b7d027====
Agent zhiu_search_agent takes 0-step Action:
{
	name: Zhipu_Search
	params: {'query': '2024诺贝尔奖得主'}
}
current_date: 2025-01-19
Observation: 2024年诺贝尔奖的获奖者如下：

1. **生理学或医学奖**：美国科学家维克托·安布罗（Victor Ambros）和加里·鲁夫坎（Gary Ruvkun）获奖，以表彰他们发现微小RNA（micr[TLDR]
Agent zhiu_search_agent takes 1-step Action:
{
	name: Finish
	params: {'response': '2024年诺贝尔奖的获奖者包括在生理学或医学、物理学、化学、文学、和平以及经济学等多个领域的杰出科学家和作家。具体获奖者名单已在上述内容中列出。'}
}
Observation: Task Completed.
=========zhiu_search_agent finish execution. TaskPackage[ID:7668636c-0cf3-481a-9f3a-eba8d1b7d027] status:
[
	completion: completed
	answer: 2024年诺贝尔奖的获奖者包括在生理学或医学、物理学、化学、文学、和平以及经济学等多个领域的杰出科学家和作家。具体获奖者名单已在上述内容中列出。
]
2024年诺贝尔奖的获奖者包括在生理学或医学、物理学、化学、文学、和平以及经济学等多个领域的杰出科学家和作家。具体获奖者名单已在上述内容中列出。


还是智谱好用